In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
math_df = pd.read_excel('/kaggle/input/contedos-matemtica-unicamp.xlsx')
math_df = math_df.replace({"Sim": 1, "Não": 0})

<ipython-input-2-d5ffec3949ae>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  math_df = math_df.replace({"Sim": 1, "Não": 0})


# **Cálculo de correlação entre a presença dos conteúdos, importante para escolher a melhor estratégia e modelo para o caso**

GEOMETRIA PLANA E PROGRESSÃO ARITIMÉTICA SEMPRE ESTÃO PRESENTES, LOGO SÃO CONSTANTES, O QUE TORNA IMPOSSÍVEL DE CALCULAR AS CORRELAÇÕES RELAIONADAS.


In [3]:
math_df_corr = math_df.drop(columns=["Ano", "Geometria Plana", "Progressão Aritmética"])
math_df_corr =math_df_corr.loc[:, math_df.var() > 0]

correlação = math_df_corr.corr()
correlação

,Geometria Espacial,Lógica,Sistemas Lineares,Função Quadrática,Conjuntos e Inequações,Análise Combinatória,Probabilidade,Função Composta e Geometria Analítica,Trigonometria,Matrizes,Grandezas e Regra de Três,Circunferência
Geometria Espacial,1.000000,-0.403390,-0.204973,0.314072,0.147573,-0.050461,0.073598,0.179196,-0.008476,0.188562,-0.012423,0.019035
Lógica,-0.403390,1.000000,-0.207870,-0.034126,-0.264575,-0.164488,-0.027067,-0.125988,0.172219,0.098601,0.252413,-0.386763
Sistemas Lineares,-0.204973,-0.207870,1.000000,-0.212814,0.054997,0.034192,0.092834,-0.054997,-0.101080,-0.474342,0.138889,0.007094
Função Quadrática,0.314072,-0.034126,-0.212814,1.000000,-0.084270,0.272434,0.258630,0.084270,0.154881,0.309570,0.007094,0.326087
Conjuntos e Inequações,0.147573,-0.264575,0.054997,-0.084270,1.000000,-0.046421,0.007161,0.033333,-0.128654,-0.372678,0.054997,0.102328
Análise Combinatória,-0.050461,-0.164488,0.034192,0.272434,-0.046421,1.000000,0.280484,0.046421,-0.121310,-0.116775,-0.177799,0.272434
Probabilidade,0.073598,-0.027067,0.092834,0.258630,0.007161,0.280484,1.000000,0.214834,0.241843,0.080064,0.354459,0.258630
Função Composta e Geometria Analítica,0.179196,-0.125988,-0.054997,0.084270,0.033333,0.046421,0.214834,1.000000,0.128654,-0.089443,-0.054997,-0.102328
Trigonometria,-0.008476,0.172219,-0.101080,0.154881,-0.128654,-0.121310,0.241843,0.128654,1.000000,0.047946,0.290604,-0.145201
Matrizes,0.188562,0.098601,-0.474342,0.309570,-0.372678,-0.116775,0.080064,-0.089443,0.047946,1.000000,0.070273,-0.107676


# **APLICAÇÃO E TESTE DO PRIMEIRO MODELO: REDE NEURAL DO TIPO RECORRENTE**

PRIMEIRO PASSO: MODELAGEM DOS DADOS


In [4]:
def formatar_dados(df, janela):
    X = []
    y = []

    for i in range(len(df) - janela):
        X.append(df.iloc[i:i + janela, 1:].values)
        y.append(df.iloc[i + janela, 1:].values)
    
    return np.array(X), np.array(y)

In [43]:
x,y = formatar_dados(math_df,5)

In [6]:
input_shape = x.shape

In [7]:
print(input_shape)

(26, 5, 15)


Definição de parâmetros:
 *  Otimizador: Adaptive Moment Estimation (Adam)
 *  Função de perda: Binary Cross Entropy
 *  Métrica: Acurácia
 *  Função de Ativação: SIGMOIDE (CAMADA DE OUTPUT) e Unidade LInear Retificada-RELU (CAMADAS OCULTAS)

In [8]:
from tensorflow import keras
from tensorflow.keras import layers

In [40]:
deep_model = keras.Sequential([
    layers.LSTM(30, activation='relu', return_sequences=True, input_shape=(5, 15)),
    layers.LSTM(28, activation='relu', return_sequences=True,),
    layers.LSTM(26, activation='relu', return_sequences=False,),
    layers.Dense(15,activation='sigmoid'),
])

shallow_model = keras.Sequential([
    layers.LSTM(32, input_shape=(3, 15), activation='relu',return_sequences=False),
    layers.Dense(15, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [41]:
deep_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

shallow_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [17]:
deep_model.summary()
shallow_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                        │ (None, 5, 30)               │           5,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 5, 28)               │           6,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 26)                  │           5,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 15)                  │             405 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,253 (71.30 KB)

 Trainable params: 18,253 (71.30 KB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                       │ (None, 32)                  │           6,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,177 (24.13 KB)

 Trainable params: 6,177 (24.13 KB)

 Non-trainable params: 0 (0.00 B)

Não dividirei o conjunto de dados entre treino e validação, pois a sequência de dados é pequena e limitada.

In [44]:
shallow_model.fit(
    x=x,
    y=y,
    batch_size=12,
    epochs=200,
)

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0000e+00 - loss: 0.7173
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.7073 
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.6967 
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.6880 
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.6795 
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.6697 
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.6606 
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.6468 
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.6369 
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.6220 
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.6030 
Epoch 12/200
3/3 ━━━━━━━━━━━━━━

In [19]:
deep_model.fit(
    x=x,
    y=y,
    batch_size=12,
    epochs=200,
)

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.0000e+00 - loss: 0.6932
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0505 - loss: 0.6899 
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.6866 
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.6833 
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.6802 
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.6754 
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.6695 
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.6631 
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.6535 
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0000e+00 - loss: 0.6391 
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.6194 
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━

Em 200 épocas, os números se tornam melhores até o final do treinamento, no entanto, a acurácia é baixa, e também pode representar um possível overfit.

In [21]:
last_5 = math_df.drop(labels=['Ano'],axis=1).tail(5)
last_5

,Geometria Plana,Geometria Espacial,Lógica,Sistemas Lineares,Função Quadrática,Conjuntos e Inequações,Análise Combinatória,Probabilidade,Progressão Aritmética,Função Composta e Geometria Analítica,Trigonometria,Matrizes,Grandezas e Regra de Três,Polinômios,Circunferência
26,1,0,1,1,0,1,1,1,1,0,1,0,1,0,0
27,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0
28,1,0,1,1,0,1,0,1,1,0,1,0,1,0,0
29,1,1,1,1,0,1,0,1,1,0,1,0,1,0,0
30,1,0,1,1,0,1,0,1,1,0,1,0,1,0,0


In [22]:
input_data = np.expand_dims(last_5.values, axis=0) 
prediction = deep_model.predict(input_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step


In [23]:
prediction

array([[1.0000000e+00, 1.0000000e+00, 7.7802050e-01, 9.9999809e-01,
        1.9531326e-01, 9.9960911e-01, 5.1748276e-02, 9.9999613e-01,
        1.0000000e+00, 1.1044377e-06, 1.0000000e+00, 2.7066790e-10,
        1.0000000e+00, 5.4775957e-34, 2.2982259e-01]], dtype=float32)

# ****Teste de um segundo modelo: Random Forest****

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [26]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)


In [27]:
math_df

,Ano,Geometria Plana,Geometria Espacial,Lógica,Sistemas Lineares,Função Quadrática,Conjuntos e Inequações,Análise Combinatória,Probabilidade,Progressão Aritmética,Função Composta e Geometria Analítica,Trigonometria,Matrizes,Grandezas e Regra de Três,Polinômios,Circunferência
0,1995,1,0,1,1,0,0,0,0,1,0,1,0,0,0,0
1,1996,1,1,1,0,0,1,0,0,1,0,1,0,0,0,0
2,1997,1,0,0,1,1,1,1,1,1,1,1,0,0,0,1
3,1998,1,1,0,1,0,1,1,1,1,0,0,0,0,0,1
4,1999,1,0,1,1,0,1,0,0,1,0,1,0,1,0,0
5,2000,1,1,1,0,1,0,0,1,1,0,1,1,1,0,0
6,2001,1,0,1,1,0,1,0,0,1,0,0,0,1,0,0
7,2002,1,1,1,1,0,1,0,1,1,1,1,0,1,0,0
8,2003,1,0,1,0,1,1,1,1,1,0,1,0,1,0,1
9,2004,1,1,0,1,0,1,0,0,1,0,1,0,1,0,0


In [29]:
def create_sliding_window(df, n_lags):
    df = df.sort_values('Ano').reset_index(drop=True)
    feature_cols = [col for col in df.columns if col != 'Ano']
    
    X_list = []
    y_list = []
    
    for i in range(n_lags, len(df)):
        features = {}
        for lag in range(1, n_lags + 1):
            for col in feature_cols:
                features[f'{col}_lag{lag}'] = df.loc[i - lag, col]
        
        X_list.append(features)
        y_list.append(df.loc[i, feature_cols].to_dict())
    
    X = pd.DataFrame(X_list)
    y = pd.DataFrame(y_list)
    
    return X, y

In [30]:
X,y  = create_sliding_window(math_df,3)

In [31]:
rf_model.fit(X,y)

RandomForestClassifier(random_state=42)

In [32]:
rf_model.score(X,y)

1.0

In [33]:
last_3 = X.tail(3)


In [34]:
last_3

,Geometria Plana_lag1,Geometria Espacial_lag1,Lógica_lag1,Sistemas Lineares_lag1,Função Quadrática_lag1,Conjuntos e Inequações_lag1,Análise Combinatória_lag1,Probabilidade_lag1,Progressão Aritmética_lag1,Função Composta e Geometria Analítica_lag1,...,Conjuntos e Inequações_lag3,Análise Combinatória_lag3,Probabilidade_lag3,Progressão Aritmética_lag3,Função Composta e Geometria Analítica_lag3,Trigonometria_lag3,Matrizes_lag3,Grandezas e Regra de Três_lag3,Polinômios_lag3,Circunferência_lag3
25,1,1,1,1,1,1,1,1,1,0,...,1,1,1,1,0,1,0,1,0,1
26,1,0,1,1,0,1,0,1,1,0,...,1,1,1,1,0,1,0,1,0,0
27,1,1,1,1,0,1,0,1,1,0,...,1,1,1,1,0,1,0,1,0,0


In [35]:
y_future_pred = rf_model.predict(last_3)
print(y_future_pred)

[[1 0 1 1 0 1 0 1 1 0 1 0 1 0 0]
 [1 1 1 1 0 1 0 1 1 0 1 0 1 0 0]
 [1 0 1 1 0 1 0 1 1 0 1 0 1 0 0]]


In [36]:
split_idx = int(len(X) * 0.8)

X_train = X.iloc[:split_idx]
X_test = X.iloc[split_idx:]

y_train = y.iloc[:split_idx]
y_test = y.iloc[split_idx:]

In [37]:
rf_model.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

In [38]:
rf_model.score(X_train,y_train)

1.0

In [39]:
rf_model.score(X_test,y_test)

0.3333333333333333